# WebScrape Turo

Business Problem: A client has purchase a new car for his fleet to be passively managed Maimi, Florida. He would like to see how his car is doing within the 6 months it's been on the market and what his competitors may look like.


READ ME:
Google Chrome and Chromedriver will need to downloaded. The verison of Chromedriver must match in order for Selenium to run. For this project, I will be using this verison:
    Google Chrome Version 100.0.4896.75 (Official Build) (x86_64)
    Chromedriver Verison ChromeDriver 100.0.4896.60
    Python 3



In [ ]:
# Install selenium to webscrape
!pip install selenium

In [ ]:
# Libraries needed for basic web-scraping
import urllib # package required to interact with live webpage
import pandas as pd # will use to store the data from the webpage

In [ ]:
# Url page that you want to webscrape (selenium)
urlpage = 'https://turo.com/us/en/search?defaultZoomLevel=11&delivery=true&deliveryLocationType=airport&endDate=04%2F12%2F2022&endTime=10%3A00&isMapSearch=false&itemsPerPage=200&latitude=39.49959&location=RNO%20-%20Reno-Tahoe%20International%20Airport&locationType=AIRPORT&longitude=-119.7681&placeId=ChIJF0m8rXA_mYARQFmvIP8eQBA&sortType=RELEVANCE&startDate=04%2F08%2F2022&startTime=10%3A00&useDefaultMaximumDistance=true'

# Run Chrome webdriver from executable path from computer
# Make sure Chromedriver matches googlechrome verison 
driver = webdriver.Chrome(executable_path="/Users/lenapham/Desktop/chromedriver")


In [ ]:
# Creating forloop to filter search page

# Get web page
driver.get(urlpage)

# Dictionary of final cars
final = {}


# Create wait time
search_grid = WebDriverWait(driver,timeout=500).until(lambda d: d.find_element(By.CSS_SELECTOR, '.schumacherGrid .ReactVirtualized__Grid'))

# Get ht of window?
window_ht = 0


# Creating forloop to pull webpage, go through every car on the list and collect the desired information
cont_search = True
while cont_search:
    car_list = search_grid.find_elements(By.CSS_SELECTOR, '.searchResult')
    for car in car_list:
        car_dict = {}
        # Create columns for name, price/day, and unique url
        car_dict['name'] = car.find_element(By.CSS_SELECTOR,'.css-1xhwfei-StyledText').text
        car_dict['price/day'] = car.find_element(By.CSS_SELECTOR, '.css-g36krj-StyledText').text
        car_dict['url'] = car.find_element(By.CSS_SELECTOR, 'a').get_attribute("href")
        # Store the ID of the original window
        original_window = driver.current_window_handle
        # Opens a new tab & switches to new tab
        driver.switch_to.new_window('tab')
        # Open tab of each car url
        driver.get(car_dict['url'])
        
        car_details = driver.find_element(By.CSS_SELECTOR,'.vehicleDetails-basicBadges').text
        # From each of those url pull information on mpg, doors, seats, and fuel type 
        mpg = car_details.find('MPG')
        if mpg > 0:
            car_dict['mpg'] = car_details[mpg-3:mpg-1]

        doors = car_details.find('door')
        if doors > 0:
            car_dict['doors'] = car_details[doors-2:doors-1]

        seats = car_details.find('seats')
        if seats > 0:
            car_dict['seats'] = car_details[seats-2:seats-1]

        fuel = re.search('(Gas)|(Electric)|(Hybrid)',car_details)
        if fuel is not None:
            car_dict['fuel_type'] = fuel.group()
        # Close the tab 
        driver.close()
        #Switch back to the orginial tab and pick another car 
        driver.switch_to.window(original_window)
        print(car_dict)
        final[car_dict['url']] = car_dict
    driver.execute_script('arguments[0].scrollIntoView()',car_list[-1])
    sleep(4)
    end_car_list = driver.find_element(By.CSS_SELECTOR,'.searchResultsEmptyState')
    cont_search = end_car_list.rect["y"]> window_ht
    window_ht = end_car_list.rect['y']
    #print(window_ht, end_car_list.rect['y'], cont_search)

print(final.values())



In [ ]:
# Puts the dictionary into pandas dataframe
df = pd.DataFrame(final.values())
print(df)

In [ ]:
# Check types, change types to str for cleaning
df.dtypes
df = df.astype(str)

#(\d+) Extracting a numerial digit more than one character '\w+@\w'
df['price/day'] = df['price/day'].str.extract('(\d+)')
print(df)

In [ ]:
# New column for vehicle_type extracted from url
df['vehicle_type'] = df['url'].str.extract('(truck|car|minivan|suv|van)-rental')

# New column for competitors from vehicle names
df['competitors'] = df['name'].str.extract('(Jeep Wrangler|Audi Q7|Honda Civic|BMW X5|Honda Accord|)')

# New column for year model of vehicle
df['year'] = df['name'].str.extract('(2005|2006|2007|2008|2009|2010|2011|2012|2013|2014|2015|2016|2017|2018|2019|2020|2021|2022)')

df.head(1000)


In [ ]:
# Extract year from the end of name 
df['name'] = df['name'].str.slice(stop=-5)
print(df)

In [ ]:
# Save df into a CSV file & export 
newdf = df.to_csv('Turo_April.csv')
print(newdf)